# 包含所有有理数的的二叉树


## rational_depth

书上给出了实现。

In [74]:
def descendants(nod):
    return [[nod[0]+nod[1],nod[1]],[nod[0],nod[0]+nod[1]]]

def rational_depth(k):
    root_list = [[1,1]]
    final_list = root_list

    for m in range(0,k):
        list_before = final_list
        final_list = list_before*2

        lglist = len(list_before)
        for i in range(0,lglist):
            descd = descendants(list_before[i])
            final_list[2*i] = descd[0]
            final_list[2*i+1] = descd[1]

    return final_list


print(rational_depth(4))

# 测试运行时间
from timeit import timeit

print(timeit(lambda: rational_depth(8),number = 10000),'s')

[[5, 1], [4, 5], [7, 4], [3, 7], [8, 3], [5, 8], [7, 5], [2, 7], [7, 2], [5, 7], [8, 5], [3, 8], [7, 3], [4, 7], [5, 4], [1, 5]]
1.7280767000002015 s


### 充分利用python的语法，可以简化表达

书上的实现方式，c++味道太浓了，我们有更简洁的方式。

根据timeit模块的计时结果（重复调用函数取最快记录），新的写法快0.4s左右，当k=8。

In [75]:
from itertools import chain
def _descendants(nod):
    """
    nod: Iterable,
    len(nod) == 2
    """
    return (nod[0]+nod[1],nod[1]),(nod[0],nod[0]+nod[1])

def rational_depth_beta(k:int):
    l = ((1,1),)
    for i in range(k):
        _l = (_descendants(nod) for nod in l)
        l = list(chain.from_iterable(_l))
    return l

print(rational_depth_beta(4))
from timeit import timeit

print(timeit(lambda: rational_depth_beta(8),number = 10000),'s')

[(5, 1), (4, 5), (7, 4), (3, 7), (8, 3), (5, 8), (7, 5), (2, 7), (7, 2), (5, 7), (8, 5), (3, 8), (7, 3), (4, 7), (5, 4), (1, 5)]
1.2892112999998062 s


用的到的知识：

* Tuple元组：python函数返回多个由逗号间隔的值，其类型为元组。元组可以当做一个“不可变数组“，就像c++中的const array.
* 使用itertools中提供的工具，简化循环的表达。itertools的工具是python标准库的一部分。这里使用的chain.from_iterable，可以将多个迭代器的输出串连起来，做为一个迭代器。举个例子：  
chain.from_iterable(['ABC', 'DEF']) --> A B C D E F
* 迭代器：数组、字符串、list、dict等等数据结构，凡是提供了一种遍历其所有元素的方法的，都被视作迭代器。如果你写了一个二叉树，实现了中序遍历，这也是一个迭代器。在代码上讲，实现了__getitem__或__iter__方法的类，都是迭代器。常见使用方式是`for item in iterator`。 
* timeit模块，python标准库。用于测试小段代码的运行速度。

## Q1 Proof

从二叉树左右子代的生成过程来看，这是显然的事情。

## Q2 script

In [76]:
def ascendant(nod):
    """
    假定输入的点是树上的点，即分子分母互质。
    """
    k,m = nod
    if k==m:
        return nod
    elif k>m:
        return [k-m,m]
    else:
        return [k,m-k]

## Q3 `ascendant_path(nod)`

In [77]:
def ascendant_path(nod):
    l = [nod]
    while nod[0] != nod[1]:
        nod = ascendant(nod)
        l.append(nod)
    return l

print(ascendant_path([5,8]))

[[5, 8], [5, 3], [2, 3], [2, 1], [1, 1]]


## Q4 depth_node


从一个内部节点`[k,m]`出发，连续取n次左子代，得到的节点是`[k+n*m,m]`，所以使用整除和余数可以更快求深度，相比于遍历先代求序列长度而言。


In [78]:
def _f(p,q):
    """
    Suppose p!=q
    """
    if p>q:
        d = p//q
        p = p%q
    else:
        d = q//p
        q = q%p
    return p,q,d

def depth_node(nod):
    d = 0
    p,q = nod
    while p!=0 and q!=0:
        p,q,_d = _f(p,q)
        d+=_d

    return d-1


print(depth_node([5,8]))


4


## Q5 180/336

其分母分子不互质，[180,336]不是树上的节点，要先化简。

另外，先代列表里有根节点，而根节点的 depth 是0。

In [79]:
def _g(p,q):
    """
    Suppose p!=q

    Return p>q
    """
    if p>q:
        r = p%q
        p = q
        q = r

    else:
        r = q%p
        p = p
        q = r
    return p,q

def gcd(p,q):
    p,q = nod
    while q!=0:
        p,q = _g(p,q)
    return p



In [80]:
nod = [180,366]
c = gcd(*nod)
nod = [v//c for v in nod ]
asc_l = ascendant_path(nod)
print("asc_l = ",ascendant_path(nod))
print("length of asc_l = ",len(ascendant_path(nod)))
print("depth = ",depth_node(nod))

asc_l =  [[30, 61], [30, 31], [30, 1], [29, 1], [28, 1], [27, 1], [26, 1], [25, 1], [24, 1], [23, 1], [22, 1], [21, 1], [20, 1], [19, 1], [18, 1], [17, 1], [16, 1], [15, 1], [14, 1], [13, 1], [12, 1], [11, 1], [10, 1], [9, 1], [8, 1], [7, 1], [6, 1], [5, 1], [4, 1], [3, 1], [2, 1], [1, 1]]
length of asc_l =  32
depth =  31


## Q6 使rational_depth递归化

比书实现的循环版本略慢，0.01s

In [81]:
from itertools import chain
def _row(d,k):
    """
    Return the k-th node at depth d."""
    if d==0: # root
        return (1,1)
    elif k%2==0: # left child
        parent =  _row(d-1,k//2)
        return parent[0]+parent[1],parent[1]
    else: # right child
        parent = _row(d-1,(k-1)//2)
        return parent[0],parent[0]+parent[1]

print(rational_depth_beta(4))

# 测试运行时间
from timeit import timeit

print(timeit(lambda: rational_depth(8),number = 10000),'s')

[(5, 1), (4, 5), (7, 4), (3, 7), (8, 3), (5, 8), (7, 5), (2, 7), (7, 2), (5, 7), (8, 5), (3, 8), (7, 3), (4, 7), (5, 4), (1, 5)]
1.7442528000001403 s


## Appendix

![hw1](images/hw/1.jpg)
![hw1](images/hw/2.jpg)
![hw1](images/hw/3.jpg)
